In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
                        SELECT 
                            *
                        FROM 
                            fbref_match_summary_v2 f
                       """, conn)

df

,team,is_home,match_date,division,season,match_url,summary_minutes,summary_goals,summary_assists,summary_pens_made,...,opp_defense_challenge_tackles,opp_defense_challenges,opp_defense_challenge_tackles_pct,opp_defense_challenges_lost,opp_defense_blocked_shots,opp_defense_blocked_passes,opp_defense_tackles_interceptions,opp_defense_clearances,opp_defense_errors,opp_keeper_psxg
0,Tottenham,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/e4bb1c35/Tottenha...,990.0,1.0,0.0,1.0,...,7.0,11.0,63.6,4.0,1.0,11.0,25.0,8.0,0.0,4.1
1,Brighton,0,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/e4bb1c35/Tottenha...,990.0,4.0,1.0,1.0,...,5.0,15.0,33.3,10.0,7.0,4.0,24.0,41.0,1.0,1.5
2,Bournemouth,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/1ff370e8/Bournemo...,990.0,2.0,2.0,0.0,...,9.0,19.0,47.4,10.0,6.0,8.0,24.0,31.0,1.0,0.0
3,Leicester City,0,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/1ff370e8/Bournemo...,990.0,0.0,0.0,0.0,...,5.0,10.0,50.0,5.0,0.0,4.0,22.0,11.0,0.0,2.1
4,Newcastle Utd,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/36844e73/Newcastl...,990.0,0.0,0.0,0.0,...,7.0,12.0,58.3,5.0,4.0,2.0,27.0,23.0,0.0,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41879,Cagliari,0,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/e8e75a75/Como-Cag...,990.0,1.0,1.0,0.0,...,11.0,17.0,64.7,6.0,2.0,3.0,27.0,22.0,0.0,1.4
41880,Cittadella,1,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/99979e25/Cittadel...,990.0,4.0,1.0,2.0,...,1.0,7.0,14.3,6.0,4.0,3.0,27.0,19.0,0.0,1.4
41881,Pisa,0,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/99979e25/Cittadel...,990.0,3.0,2.0,0.0,...,5.0,13.0,38.5,8.0,2.0,3.0,25.0,13.0,0.0,2.2
41882,Parma,1,2022-08-12,Serie B,2022-2023,https://fbref.com/en/matches/be681acc/Parma-Ba...,990.0,2.0,1.0,0.0,...,5.0,16.0,31.3,11.0,1.0,3.0,24.0,22.0,0.0,2.1


In [4]:
for col in df.select_dtypes(include=['object']).columns:
    # Try converting a sample
    sample_converted = pd.to_numeric(df[col].dropna().head(), errors='coerce')
    if not sample_converted.isna().all():
        print(f"Column '{col}' can be converted to numeric")
        df[col] = pd.to_numeric(df[col], errors='coerce')

Column 'summary_minutes' can be converted to numeric
Column 'summary_assists' can be converted to numeric
Column 'summary_pens_made' can be converted to numeric
Column 'summary_pens_att' can be converted to numeric
Column 'summary_tackles' can be converted to numeric
Column 'summary_interceptions' can be converted to numeric
Column 'summary_blocks' can be converted to numeric
Column 'summary_sca' can be converted to numeric
Column 'summary_gca' can be converted to numeric
Column 'summary_passes_completed' can be converted to numeric
Column 'summary_passes' can be converted to numeric
Column 'summary_passes_pct' can be converted to numeric
Column 'summary_progressive_passes' can be converted to numeric
Column 'summary_carries' can be converted to numeric
Column 'summary_progressive_carries' can be converted to numeric
Column 'summary_take_ons' can be converted to numeric
Column 'summary_take_ons_won' can be converted to numeric
Column 'possession_minutes' can be converted to numeric
Col

In [9]:
# Quick correlation screening
numeric_df = df.select_dtypes(include=['number'])
correlations = numeric_df.corr()["pass_types_corner_kicks"].abs().sort_values(ascending=False)
top_features = correlations
top_features = top_features[top_features > 0.5]

top_features

pass_types_corner_kicks                   1.000000
passing_corner_kicks                      1.000000
passing_corner_kicks_in                   0.759521
pass_types_corner_kicks_in                0.759521
pass_types_crosses                        0.682711
misc_crosses                              0.682711
passing_crosses                           0.682711
possession_touches_att_pen_area           0.627374
possession_touches_att_3rd                0.622814
summary_progressive_passes                0.581169
possession_progressive_passes_received    0.580000
opp_possession_touches_def_pen_area       0.578577
summary_shots                             0.549833
summary_sca                               0.547465
opp_defense_clearances                    0.536762
opp_pass_types_passes                     0.524752
opp_summary_passes                        0.524752
opp_passing_passes                        0.524752
opp_defense_blocked_shots                 0.524152
opp_passing_passes_live        

In [19]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Start with all numeric features
X_train, X_test, y_train, y_test = train_test_split(numeric_df.drop(['pass_types_corner_kicks', 'passing_corner_kicks', 
                                                                     'pass_types_corner_kicks_in', 'passing_corner_kicks_in'], axis=1), 
                                                   numeric_df['pass_types_corner_kicks'], 
                                                   test_size=0.2, random_state=42)

# Basic XGBoost with built-in regularization
model = xgb.XGBRegressor(
   n_estimators=100,
   max_depth=6,
   reg_alpha=0.1,  # L1 regularization
   reg_lambda=1.0, # L2 regularization
   random_state=42
)

model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Simple evaluation
print("Model Performance:")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"Mean target value: {y_test.mean():.4f}")

# Get feature importance after training
importance = pd.DataFrame({
   'feature': X_train.columns,
   'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(importance.head(25))

Model Performance:
R² Score: 0.5233
RMSE: 2.1049
Mean target value: 5.0921

Top 10 Most Important Features:
                                  feature  importance
57                           misc_crosses    0.231317
32             possession_touches_att_3rd    0.062065
33        possession_touches_att_pen_area    0.061697
120                   opp_summary_touches    0.039742
129          opp_summary_passes_completed    0.030688
130                    opp_summary_passes    0.025842
178            opp_pass_types_passes_live    0.020379
69                 pass_types_passes_dead    0.018821
128                       opp_summary_gca    0.017037
126                 opp_summary_xg_assist    0.015806
45   possession_carries_into_penalty_area    0.014469
75            pass_types_corner_kicks_out    0.014073
188  opp_pass_types_corner_kicks_straight    0.013567
123                    opp_summary_blocks    0.013521
26                   summary_take_ons_won    0.013335
34           possession_touc